### 引入相關python模組

In [2]:
m = tf.keras.applications.InceptionV3(include_top=False, input_shape=(299,299,3))
m.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d_89 (Conv2D)              (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_89 (BatchNo (None, 149, 149, 32) 96          conv2d_89[0][0]                  
__________________________________________________________________________________________________
activation (Activation)         (None, 149, 149, 32) 0           batch_normalization_89[0][0]     
_______________________________________________________________________________________

In [1]:
from mycnn import InceptionV3
from mycnn import utils
import tensorflow as tf
import cv2
import numpy as np
import matplotlib.pyplot as plt

inception = InceptionV3(classes_num=2)
inception.summary()

Model: "InceptionV3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image (InputLayer)           [(None, 299, 299, 3)]     0         
_________________________________________________________________
conv1_conv (Conv2D)          (None, 149, 149, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 149, 149, 32)      96        
_________________________________________________________________
conv1_relu (ReLU)            (None, 149, 149, 32)      0         
_________________________________________________________________
conv2_conv (Conv2D)          (None, 147, 147, 32)      9216      
_________________________________________________________________
conv2_bn (BatchNormalization (None, 147, 147, 32)      96        
_________________________________________________________________
conv2_relu (ReLU)            (None, 147, 147, 32)      

### 使用Keras API來建立Dataset實例

利用Keras API中的`preprocessing`模組的`image_dataset_from_directory`  
用此函數來建立貓狗的資料集，此函數將會回傳`tf.data.Dataset`的實例  
接著使用`map`函式來重新縮放(正規化)資料區間至 [0, 1]

#### tf.keras.preprocessing.image_dataset_from_directory

```
參數名稱            型態    說明
directory        : str   : 資料路徑 (子資料夾為類別)
image_size       : tuple : 影像大小
batch_size       : int   : 批次大小
label_mode       : str   : 標記模式 "categorical" (註:其他模式需要修改loss函數)
validation_split : float : 分離驗證集的比例
subset           : str   : 選擇訓練集 "training"、驗證集 "validation"
seed             : int   : 亂數種子
```

In [ ]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    r'D:\Datasets\DogsVsCats\train',
    image_size=(224,224),
    batch_size=20,
    label_mode="categorical",
    validation_split=0.2,
    subset="training",
    seed=10
)
train_file_paths = train_dataset.file_paths
train_dataset.map(lambda x, y: (x/255., y))


valid_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    r'D:\Datasets\DogsVsCats\train',
    image_size=(224,224),
    batch_size=20,
    label_mode="categorical",
    validation_split=0.2,
    subset="validation",
    seed=10
)
valid_file_paths = valid_dataset.file_paths
valid_dataset.map(lambda x, y: (x/255., y))

### 載入GoogLeNet模型

```
參數名稱       型態    說明
input_shape : tuple : 輸入影像形狀
classes_num : int   : 輸出類別數量
```

In [ ]:
inception = InceptionV3(classes_num=2)
inception.summary()

### 配置訓練參數

```
參數名稱      型態                         說明
logdir     : str                        : 儲存路徑
epochs     : int                        : 訓練次數
batch_size : int                        : 批次大小 (註:此設定需與image_dataset_from_directory的批次大小一致)
optimizer  : str or tf.keras.optimizers : 優化函數
loss       : str or tf.keras.loss       : 損失函數
metrics    : list                       : 評估函數清單
```

In [ ]:
inception.setup_training(
    'log_googlenet',
    epochs=10,
    batch_size=20,  # batch size depend on `image_dataset_from_directory`
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
inception.add_callback(
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', 
        factor=0.1, patience=2,
        min_lr=0.00001,
        verbose=1
    )
)

### 開始訓練

輸入參數分別為訓練資料集、驗證資料集的實例

In [ ]:
inception.train_dataset(train_dataset, valid_dataset)

### 繪製訓練過程曲線

可以用來確認權重是否有收斂的趨勢、檢查是否有過擬合狀況

In [ ]:
inception.show_history(["loss", "accuracy"])

### 使用測試資料來確認模型對於新資料的效能

In [ ]:
inception.eval_dataset(valid_dataset)

### 使用confusion matrix來更進一步確認分類性能

- 預測測試資料的分數 (基於softmax函數計算機率分布)
- 使用`argmax`將分數轉成類別ID
- 輸出分類報告 (印出confusion matrix、分類報告；輸出完整報表)
- 繪製confusion matrix，分為recall、precision

> Note:  
recall: 召回率，在所有GT中，真正預測出TP的指標  
precision: 精確率，在所有預測結果中，真正為TP的指標  
(GT: 真實情況；TP: 正樣本)

In [ ]:
pred_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    r'D:\Datasets\DogsVsCats\train',
    image_size=(224,224),
    batch_size=20,
    label_mode="categorical",
    validation_split=0.2,
    subset="validation",
    seed=10
)
pred_dataset = pred_dataset.map(lambda x, y: (x/255., y))

pr_score = None
pr_label = None
gt_label = None
for ind, batch_set in enumerate(pred_dataset):
    batch_im, batch_gt = batch_set
    batch_pr = inception.pred(batch_im.numpy())
    if ind == 0:
        pr_score = batch_pr
        pr_label = batch_pr.argmax(axis=-1)
        gt_label = batch_gt.numpy().argmax(axis=-1)
    else:
        pr_score = np.concatenate([pr_score, batch_pr])
        pr_label = np.concatenate([pr_label, batch_pr.argmax(axis=-1)])
        gt_label = np.concatenate([gt_label, batch_gt.numpy().argmax(axis=-1)])

target_names = ["Cats", "Dogs"]

report = utils.export_classification_report(
    gt_label, pr_label, pr_score,
    target_names=target_names,
    logpath=inception.logdir
)

cm = report["confusion_matrix"]
cm_precision = cm/cm.sum(axis=0)
cm_recall = cm/cm.sum(axis=1)
utils.plot_confusion_matrix(cm_recall, target_names, inception.logdir, title='Confusion Matrix (recall)')
utils.plot_confusion_matrix(cm_precision, target_names, inception.logdir, title='Confusion Matrix (precision)')